# **TANGOS: The Agile Numerical Galaxy Organisation System**


## Step 1: Installation
To install: <br>
\>>git clone https://github.com/anchwr/tangos.git <br>
\>>cd tangos <br>
\>>python setup.py develop <br>
This will install my fork of tangos, which works with Enzo outputs. You can install the public version of tangos by typing <br>
\>>pip install tangos <br>
but you won't be able to use certain functions (e.g., calculate_all) that need access to the handler class that was used to create the database. <br>
More detailed instructions can be found at: https://pynbody.github.io/tangos/index.html <br>

## Step 2: Download a database
Basic database containing info for Tempest RDs can be found at: https://github.com/foggie-sims/foggie/blob/master/foggie/halo_infos/008508/nref11c_nref9f/Tempest.9f11c.RD.db <br>(Size as of 8/2/2021: ~10 MB) <br> <br>
You'll also need to tell tangos where to find your database. Add the following lines to your .bashrc (.bash_profile, etc):<br>
export TANGOS_SIMULATION_FOLDER = /path/to/folder/containing/tangos/database/ <br>
export TANGOS_DB_CONNECTION = /path/to/folder/containing/tangos/database/Tempest.9f11c.RD.db <br>
So, for me, this is:<br>
export TANGOS_SIMULATION_FOLDER=/Users/Anna/Research/Outputs/tangostest/ <br>
export TANGOS_DB_CONNECTION=/Users/Anna/Research/Outputs/tangostest/Tempest.9f11c.RD.db


## Step 3: Check that you can load the database
(Source your .bashrc and reload this notebook if necessary) <br>


In [ ]:
import tangos as db

Which simulations do we have access to? If all went well, you should see <Simulation("Tempest.9f11c.RD")>

In [ ]:
db.all_simulations()

Load the Tempest database:

In [ ]:
sim = db.get_simulation('Tempest.9f11c.RD')

Check that you can see all of the timesteps (you should see a list showing RD0012-RD0042):

In [ ]:
sim.timesteps

Let's look at RD0042. There are a couple of ways to select an individual timestep. My preferred way is just via indexing. There are 31 timesteps here total, so the most recent is:

In [ ]:
sim[30]

However, if you know the name of the timestep, but not its index, you can also access it that way:

In [ ]:
db.get_timestep('Tempest.9f11c.RD/%42')

We can access individual halos in much the same way:

In [ ]:
sim[30][1]

and

In [ ]:
db.get_halo('Tempest.9f11c.RD/%42/halo_1')

are equivalent. Tangos automatically sorts halos by the number of dark matter particles that they possess, so halo 1 is the most massive halo, halo 2 is the next most massive, and so on.

## Step 4: Look at what properties are actually stored in the database

The utility of tangos comes from the fact that you can calculate the same properties for every halo once, store them, and then rapidly access those properties without having to load the simulation itself into yt (or pynbody). You can check which properties exist for a given halo:

In [ ]:
sim[30][1].keys()

These will vary depending on which halo finder was used on the original simulation and which custom properties your db creator has stored in the database. In this case, rockstar and consistent-trees have been used to find the halos and link them across timesteps. Many of the properties (e.g., Mvir, Center) come directly from the rockstar halo catalogs (albeit converted into physical units). Other properties (e.g., M_star, ContamFrac) are custom. Descriptions of each property and the relevant units are available at: https://github.com/foggie-sims/foggie/blob/master/foggie/halo_infos/008508/nref11c_nref9f/tangos.README.txt . I will do my best to keep this up to date!

We can calculate a single property for a single halo:

In [ ]:
sim[30][1]['M_coldgas']

Or we can calculate multiple properties for every halo in this timestep simultaneously:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams['xtick.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 20
plt.rcParams['axes.titlesize'] = 20

Mh, Ms = sim[30].calculate_all('Mvir','M_star')

plt.figure(figsize=(8,8))
plt.plot(np.log10(Mh),np.log10(Ms),'k.',markersize=13)
plt.xlabel('log$_{10}$(M$_\mathrm{vir}$/M$_\odot$)',fontsize=20)
plt.ylabel('log$_{10}$(M$_\star$/M$_\odot$)',fontsize=20)
plt.xlim(6,12)
plt.ylim(2,11)
plt.show()

Something to note: when we call "calculate_all", tangos processes the halos in the order they were read in from the halo catalog, NOT the order they are in the database. Each halo has a few different IDs. The important ones here are halo_number (the number of the halo in the database) and finder_id (the number of the halo in the rockstar halo catalog). You can see that these differ pretty drastically:

In [ ]:
print (sim[30][1].halo_number)
print (sim[30][1].finder_id)

Because of this, it's a good idea to explicitly tell tangos to sort by halo_number if you decide to call calculate_all but still want to be able to associate the resultant quantities with individual halos:

In [ ]:
sim[22].calculate_all('M_gas',order_by_halo_number=True)

In addition to the properties listed via halo.keys(), there may be "live calculations" available. These are properties that the database doesn't explicitly store, but which can be calculated entirely from existent properties. There are a number that are available by default (a full list can be found at https://pynbody.github.io/tangos/live_calculation.html), but your db creator may also have made some custom live calculations. Because these can be called at any time (and therefore don't need to have been available at the time that the db was created), they should work on any database you have access to so long as the necessary properties are present. For instance, my branch of tangos includes live calculations to identify satellites and hosts:

In [ ]:
sats = sim[30][2].calculate('Satellites()')
print (sats)

This calculation requires only the 'Center', 'Rvir', and 'Mvir' properties. Note that what's been returned is actually an array of halo objects. The properties of the halos are directly accessible through these, so the following two lines are equivalent:

In [ ]:
sats[1].calculate('Hosts()')

In [ ]:
sim[30][5].calculate('Hosts()')

## Step 5: Travel through time

Tangos links halos across time either through its own bridging program or by reading in output from a merger tree. This allows us to calculate properties for halos at multiple points in time. We can go just one timestep back:

In [ ]:
sim[30][3].calculate('earlier(1).Mvir')

Or we can look at the evolution of this property over every available timestep:

In [ ]:
sim[30][3].calculate_for_progenitors('Mvir')

Note: the value for the timestep you started with (RD0042) is the first entry while the value for the earliest timestep this halo is traced to (RD0021) is the last entry.

We can also go forwards in time:

In [ ]:
mv, t = sim[1][18].calculate_for_descendants('Mvir','t()')
plt.figure(figsize=(8,8))
plt.plot(t,np.log10(mv),'k-')
plt.xlim(0,14)
plt.ylim(8.4,9.6)
plt.xlabel('Time (Gyr)',fontsize=20)
plt.ylabel('log$_{10}$(M$_\mathrm{vir}$/M$_\odot$)',fontsize=20)

We can also look at how mass grows by directly looking at mergers. There are a number of ways to do this. If you just want to know if a halo experienced a merger between one timestep and the previous one, you can just look at the halo keys. If you see more than one 'consistent_trees_link' entry, there's been a recent merger!

In [ ]:
sim[30][4].keys()

Consistent_trees_link returns halo objects from the previous timestep that are part of the halo in the current timestep, so you can use it directly to access the properties of the merging halos. 

In [ ]:
sim[30][4]['consistent_trees_link']

For instance, we can find out the mass ratio of the merging halos:

In [ ]:
sim[30][4]['consistent_trees_link'][0]['Mvir']/sim[30][4]['consistent_trees_link'][1]['Mvir']

You can also grab the entire merger history of a halo at once

In [ ]:
import tangos.examples.mergers as tmer

z,rat,hobj = tmer.get_mergers_of_major_progenitor(sim[30][1])
print ('Redshifts of mergers:\n',z)
print ('Mass ratios of mergers:\n',rat)
print ('Halos involved in mergers:\n',hobj)

This is a fairly complex merger history and likely better processed visually. We can do that by launching tangos' web server:<br>
\>>tangos serve <br>
You should see something like "Serving on http://555.5.5.5:6543". Enter the address into your web browser and click on the simulation you want, followed by the timestep and the halo. Clicking on the circle next to "merger tree" will give you a visual representation of your halo's merger history. You can also plot just about every other quantity in the database against time (or any other quantity), too!

## Step 6: Use tangos and yt together

Let's go back to the halo that we traced forwards in time a few steps ago. We can see that this halo loses mass, presumably through tidal stripping. Thus far, we've done all of our analysis without accessing the actual simulation data or loading in yt. However, if we want to visualize the tidal debris, we'll need to use both tangos and yt. Using tangos, we can identify when the halo was at its maximum mass, where it was located at that time, and how large it was.

In [ ]:
hn,rvir,cen,mv = sim[1][18].calculate_for_descendants('halo_number()','Rvir','Center_cu','Mvir')
maxm = np.argmax(mv)
print (sim[int(1+maxm)][int(hn[maxm])])
r = rvir[maxm]
c = cen[maxm]

Using yt, we can read in all of the particles that belonged to the halo at this time by creating a sphere of radius r at point c.

In [ ]:
import yt
from yt.units.yt_array import YTQuantity
from yt.data_objects.particle_filters import add_particle_filter

ds = yt.load('/Volumes/Rapid/Tempest/nref9f_nref11c/RD0034/RD0034')
ad = ds.all_data()

halo = ds.sphere(center=c,radius=YTQuantity(r,'kpc'))

h_stars = halo['nbody','particle_index'][halo['nbody','particle_type']==2]
h_dm = halo['nbody','particle_index'][halo['nbody','particle_type']==4]

Now let's see where they are at z=0 with respect to Tempest:

In [ ]:
def HaloStars(pfilter, data):
    filter = np.isin(data['nbody','particle_index'],h_stars)
    return filter

add_particle_filter("halo_stars", function=HaloStars, filtered_type='nbody', \
                    requires=["particle_index"])

def HaloDM(pfilter, data):
    filter = np.isin(data['nbody','particle_index'],h_dm) 
    return filter

add_particle_filter("halo_dm", function=HaloDM, filtered_type='nbody', \
                    requires=["particle_index"])

In [ ]:
ds = yt.load('/Volumes/Rapid/Tempest/nref9f_nref11c/RD0042/RD0042')
ad = ds.all_data()

ds.add_particle_filter('halo_stars')
ds.add_particle_filter('halo_dm')

TempestCenter = sim[30][1]['Center_cu']
TempestRadius = sim[30][1]['Rvir']

GuessReg = ds.sphere(TempestCenter,radius=(500,'kpc'))

xpos_dm = YTQuantity(sim[30][1]['X']*1000,'kpc')-GuessReg['halo_dm','particle_position_x'].in_units('kpc')
ypos_dm = YTQuantity(sim[30][1]['Y']*1000,'kpc')-GuessReg['halo_dm','particle_position_y'].in_units('kpc')
zpos_dm = YTQuantity(sim[30][1]['Z']*1000,'kpc')-GuessReg['halo_dm','particle_position_z'].in_units('kpc')
xpos_stars = YTQuantity(sim[30][1]['X']*1000,'kpc')-GuessReg['halo_stars','particle_position_x'].in_units('kpc')
ypos_stars = YTQuantity(sim[30][1]['Y']*1000,'kpc')-GuessReg['halo_stars','particle_position_y'].in_units('kpc')
zpos_stars = YTQuantity(sim[30][1]['Z']*1000,'kpc')-GuessReg['halo_stars','particle_position_z'].in_units('kpc')

In [ ]:
f1 = plt.figure(figsize=(8,8))
plt.plot(xpos_dm,ypos_dm,'k.',markersize=10)
plt.plot(xpos_stars,ypos_stars,'r.',markersize=8)
Tempcir = plt.Circle((0,0),TempestRadius,color='b',fill=False)
f1.gca().add_artist(Tempcir)
plt.xlabel('x/kpc', fontsize=20)
plt.ylabel('y/kpc', fontsize=20)
plt.show()

f2 = plt.figure(figsize=(8,8))
plt.plot(ypos_dm,zpos_dm,'k.',markersize=10)
plt.plot(ypos_stars,zpos_stars,'r.',markersize=8)
Tempcir = plt.Circle((0,0),TempestRadius,color='b',fill=False)
f2.gca().add_artist(Tempcir)
plt.xlabel('y/kpc', fontsize=20)
plt.ylabel('z/kpc', fontsize=20)
plt.show()

f2 = plt.figure(figsize=(8,8))
plt.plot(xpos_dm,zpos_dm,'k.',markersize=10)
plt.plot(xpos_stars,zpos_stars,'r.',markersize=8)
Tempcir = plt.Circle((0,0),TempestRadius,color='b',fill=False)
f2.gca().add_artist(Tempcir)
plt.xlabel('x/kpc', fontsize=20)
plt.ylabel('z/kpc', fontsize=20)
plt.show()

**There are also plenty of other ways to use tangos! A more complete set of tutorials can be found at** https://pynbody.github.io/tangos/data_exploration.html